In [ ]:
import json
import requests
from llama_cpp import Llama
from typing import List, Dict, Any

# ------------- LLM RESPONSE GENERATION -------------

def generate_recipe_fallback(
    fridge_items: Dict[str, int],
    preferences: Dict[str, str],
    llm_model: Any
) -> Dict[str, Any]:
    """
    Generate a fallback recipe suggestion when no good matches are found.

    Args:
        fridge_items: Dictionary of available fridge items with quantities (e.g., {'apple': 1, 'ketchup': 3})
        preferences: Dictionary of user preferences (e.g., {'mealType': 'Dessert', 'dietaryNeeds': 'Low-carb'})
        llm_model: Loaded Llama 3.2 model

    Returns:
        Dictionary with fallback recipe information
    """
    # Prepare lists for available ingredients and dietary restrictions
    available_ingredients = list(fridge_items.keys())  # Use keys as ingredients
    dietary_restrictions = [preferences['dietaryNeeds']]  # Use dietaryNeeds from preferences

    # Create prompt for the LLM to generate a recipe
    ingredients_list = ", ".join(available_ingredients[:15])  # Limit to first 15 ingredients
    restrictions_list = ", ".join(dietary_restrictions)

    prompt = f"""
    Create a simple recipe that meets these requirements:

    Available ingredients: {ingredients_list}
    Dietary restrictions: {restrictions_list}
    User request: {preferences['mealType']} with {preferences['mainIngredient']} as the main ingredient and {preferences['dishType']} as the dish type.

    Return ONLY a JSON object with this exact format:
    {{
      "name": "Recipe name",
      "ingredients": ["ingredient1", "ingredient2", ...],
      "steps": "Step-by-step instructions",
      "tags": "comma,separated,tags",
      "notes": "Any special notes about dietary restrictions"
    }}

    The recipe should use as many of the available ingredients as possible while respecting all dietary restrictions.
    """

    # Generate response with Llama 3.2
    try:
        response = llm_model.create_completion(
            prompt,
            max_tokens=1024,
            temperature=0.7,
            top_p=0.9,
            stop=["```", "User:"]
        )

        # Parse JSON response
        response_text = response['choices'][0]['text'].strip()
        # Extract JSON if embedded in text
        if "```json" in response_text:
            json_str = response_text.split("```json")[1].split("```")[0].strip()
        elif "{" in response_text and "}" in response_text:
            json_str = response_text[response_text.find("{"):response_text.rfind("}")+1]
        else:
            json_str = response_text

        fallback_recipe = json.loads(json_str)

        # Add fallback flag
        fallback_recipe["is_fallback"] = True
        fallback_recipe["combined_score"] = 0.5  # Arbitrary middle score

        return fallback_recipe
    except Exception as e:
        # If JSON parsing fails, return a basic fallback
        print(f"Error generating fallback recipe: {str(e)}")
        return {
            "name": "Simple " + preferences['dietaryNeeds'] + " Recipe",
            "ingredients": available_ingredients[:5],
            "steps": "Combine available ingredients to taste.",
            "tags": preferences['dietaryNeeds'],
            "notes": "This is a basic recipe using your available ingredients.",
            "is_fallback": True,
            "combined_score": 0.5
        }

In [ ]:
def process_recipe_query(
    fridge_items: Dict[str, int],
    preferences: Dict[str, str]
) -> str:
    """
    Main function to process a user query about recipes.

    Args:
        fridge_items: Dictionary of available fridge items
        preferences: Dictionary of user preferences

    Returns:
        Natural language response with recipe recommendations
    """
    try:
        # Load models and data
        llm_model = Llama(
            model_path="/path/to/your/llama-3.2-8b.gguf",  # Absolute path to model
            n_ctx=4096,  # Context window
            n_gpu_layers=-1  # Use all available GPU layers if possible
        )

        # Generate fallback recipe based on the input
        fallback_recipe = generate_recipe_fallback(
            fridge_items=fridge_items,
            preferences=preferences,
            llm_model=llm_model
        )

        # Convert fallback recipe to a user-friendly response
        response = f"Here is a recipe suggestion based on your preferences and ingredients:\n\n"
        response += f"Recipe Name: {fallback_recipe['name']}\n"
        response += f"Ingredients: {', '.join(fallback_recipe['ingredients'])}\n"
        response += f"Steps: {fallback_recipe['steps']}\n"
        response += f"Tags: {fallback_recipe['tags']}\n"
        response += f"Notes: {fallback_recipe['notes']}\n"

        return response

    except Exception as e:
        error_msg = f"An unexpected error occurred: {str(e)}"
        print(error_msg)
        return "I'm sorry, I encountered an unexpected error. Please try again."

In [ ]:
fridge_items = {"apple": 1, "ketchup": 3, "cucumber": 3, "chicken": 2}
preferences = {
    "mealType": "Dinner",
    "dietaryNeeds": "Low-carb",
    "cuisineType": "None",
    "mainIngredient": "Chicken",
    "dishType": "Casserole"
}

# Process the recipe query and print the response
response = process_recipe_query(fridge_items, preferences)
print(response)